In [334]:
import pandas as pd
import os
import multiprocessing as mp
import time
from pathlib import Path
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', 100)

In [335]:
os.getcwd()

'c:\\Users\\ThibJacq\\code\\esa-data-bank_banque-donnees-ees\\Codes\\Section_04_Final_Data_Merge_and_Visualization\\bundle'

In [336]:
index_with_vcs_eng = str(Path().resolve().parents[1]) + '\\Section_05_VEC_Labeling_for_Tables\\mydata\\esa_index_keyword_labeled_2021-08-26_20-24-44.csv'
index_with_bad_csv_column = str(Path().resolve().parents[1]) + "\\Section_05_VEC_Labeling_for_Tables\\mydata\\ESA_website_ENG_2021_01_28.csv"

In [337]:
df_index_raw = pd.read_csv(index_with_vcs_eng, encoding="utf-8")
df_index_raw.drop(columns=['text'], inplace=True)
df_bad_csv_column = pd.read_csv(index_with_bad_csv_column, encoding="utf-8")
df_index_raw['bad_csv'] = df_bad_csv_column['bad_csv']


df_index_raw['Data ID'].replace(to_replace=nan, value=0, inplace=True)
df_index_raw['Data ID'].replace(to_replace="https://apps.cer-rec.gc.ca/REGDOCS/File/Download/895339", value="895339", inplace=True)
df_index_raw['bad_csv'].replace(to_replace=nan, value=bool(False), inplace=True)
df_index_raw.rename(columns={'CSV Filename': 'filename'}, inplace=True)
df_index_raw.drop(columns=['csvFileName'], inplace=True)

def df_convert_all_floats_in_column_to_int(df, col_name):
    df[col_name] = df[col_name].astype(float)
    df[col_name] = df[col_name].astype(int)

df_convert_all_floats_in_column_to_int(df_index_raw, "Data ID")
df_index_raw.drop(columns=["Unnamed: 0"], inplace=True)
df_index_raw

,Index,Title,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,Data ID,PDF Download URL,Application Type (NEB Act),Pipeline Location,Hearing order,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,ESA Section(s) Topics,CSV Download URL,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,Download folder name,Zipped Project Link,bad_csv,filename,"Landscape, terrain, and weather",Soil,Plants,Water,Fish,Wetlands,Wildlife,Species at Risk,Greenhouse gas emissions,Air emissions,Noise,Electricity and electromagnetism,Proximity to people,"Archaeological, paleontological, historical, and culturally significant sites and resources",Human access to boats and waterways,"Indigenous land, water, and air use",Impact to social and cultural well-being,Impact to human health and viewscapes,"Social, cultural, economic infrastructure and services",Economic Offsets and Impact,Environmental Obligations,Treaty and Indigenous Rights
0,9134,TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABORIGINAL FIELD STUDY PARTICIPATION FOR THE PROJECT,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/1060040,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059614,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-3-summary-of-aquatics-field-work-and-ab...,14,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-3-summary-of-aquatics-field-work-and-aborigi_pt-1_pg-14_doc-num-A3Q6H2.csv,0,0,0,17,15,0,1,0,0,0,0,0,0,0,0,26,1,0,0,0,0,38
1,9135,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG THE NORTH MONTNEY MAINLINE (AITKEN CREEK SECTION),Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/1060040,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059614,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-4-summary-of-watercourse-crossings-alon...,17,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossings-along-the_pt-1_pg-17_doc-num-A3Q6H2.csv,3,2,6,47,90,3,14,16,1,0,0,0,0,0,25,28,2,0,1,20,2,28
2,9136,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG THE NORTH MONTNEY MAINLINE (AITKEN CREEK SECTION),Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/1060040,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059614,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-4-summary-of-watercourse-crossings-alon...,18,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossings-along-the_pt-2_pg-18_doc

In [338]:
# list of VCs

labels_list = ['Landscape, terrain, and weather', 
                'Soil',
                'Plants',
                'Water',
                'Fish',
                'Wetlands',
                'Wildlife',
                'Species at Risk',
                'Greenhouse gas emissions',
                'Air emissions',
                'Noise',
                'Electricity and electromagnetism',
                'Proximity to people',
                'Archaeological, paleontological, historical, and culturally significant sites and resources',
                'Human access to boats and waterways',
                'Indigenous land, water, and air use',
                'Impact to social and cultural well-being',
                'Impact to human health and viewscapes',
                'Social, cultural, economic infrastructure and services',
                'Economic Offsets and Impact',
                'Environmental Obligations',
                'Treaty and Indigenous Rights']

labels_list_fr = ['Paysage, terrain et conditions météorologiques',
                    'Sol',
                    'Plantes',
                    'Eau',
                    'Poissons',
                    'Milieux humides',
                    'Faune',
                    'Espèces en péril',
                    'Émissions de gaz à effet de serre',
                    'Émissions atmosphériques',
                    'Bruit',
                    'Électricité et électromagnétisme',
                    'Lieux habités à proximité',
                    'Sites et ressources archéologiques, paléontologiques, historiques ou importants sur le plan culturel',
                    'Activités à bord d’embarcation ou sur des voies navigables',
                    'Utilisation des terres, de l’eau et du plein air par les Autochtones',
                    'Incidences sur le bien-être social et culturel',
                    'Incidence sur la santé humaine et le panorama',
                    'Infrastructure et services sociaux, culturels et économiques',
                    'Compensations et répercussions économiques',
                    'Obligations en matière d’environnement',
                    'Droits ancestraux et issus de traités']

df_table_id = df_index_raw.groupby(['Title', 'Data ID'])[labels_list].apply(lambda x : x.astype(int).sum())\
.reset_index().reset_index().rename(columns={'index': 'ID'})
df_table_id.head()

,ID,Title,Data ID,"Landscape, terrain, and weather",Soil,Plants,Water,Fish,Wetlands,Wildlife,Species at Risk,Greenhouse gas emissions,Air emissions,Noise,Electricity and electromagnetism,Proximity to people,"Archaeological, paleontological, historical, and culturally significant sites and resources",Human access to boats and waterways,"Indigenous land, water, and air use",Impact to social and cultural well-being,Impact to human health and viewscapes,"Social, cultural, economic infrastructure and services",Economic Offsets and Impact,Environmental Obligations,Treaty and Indigenous Rights
0,0,TABLE 1 2012 VEGETATION SURVEY LOCATIONS,894928,0,1,18,4,2,27,0,0,0,0,0,0,0,0,2,5,0,0,1,0,0,5
1,1,TABLE 1 ABORIGINAL COMMUNITIES AND THEIR DISTANCE FROM THE PROPOSED PIPELINE RIGHT-OF-WAY,600731,0,0,0,4,4,0,0,0,0,0,0,0,1,0,0,12,2,0,2,0,0,20
2,2,TABLE 1 AREAS ASSESSED UNDER PERMIT 07-368,487965,0,0,0,2,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,TABLE 1 AREAS ASSESSED UNDER PERMIT 2007-252,487871,1,0,1,11,2,1,0,1,0,0,0,0,0,2,7,0,1,1,3,0,0,0
4,4,TABLE 1 BIOSECURITY MANAGEMENT PLAN RESPONSIBILITIES,2545320,0,7,29,0,0,0,0,2,0,0,2,0,1,0,0,0,0,0,18,13,4,0


In [339]:
df_table_id.shape

(15657, 25)

In [340]:
# Create a temporary column in the index dataframe as table identification
df_table_id = df_index_raw.groupby(['Title', 'Data ID'])[labels_list].apply(lambda x : x.astype(int).sum())\
.reset_index().reset_index().rename(columns={'index': 'ID'})
df_index_raw = df_index_raw.merge(df_table_id, left_on=['Title', 'Data ID'], right_on=['Title', 'Data ID'], suffixes=('_x', ''))
df_index_raw.drop(df_index_raw.filter(regex='_x$').columns, axis=1, inplace=True)


In [341]:
df_index_raw.shape

(29912, 56)

In [342]:
# Remove bad csvs
df_index = df_index_raw[~df_index_raw['bad_csv']]

In [343]:
assert len(df_index[df_index['bad_csv'] == True]) == 0

In [344]:
# Add a new column - Project Download Path
df_index['Project Download Path'] = df_index['Download folder name']\
    .apply(lambda x: '/projects/{}.zip'.format(x))

<ipython-input-344-f03c43fac486>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_index['Project Download Path'] = df_index['Download folder name']\


In [345]:
# Add a new column - Table Download Path
df_table_filename = df_index.sort_values(['PDF Page Number'])\
    .groupby('ID')['filename'].first().reset_index().rename(columns={'filename': 'Table Name'})
df_table_filename['Table Name'] = df_table_filename['Table Name']\
    .apply(lambda x: x.replace('.csv', '').replace('--', '-'))
df_index = df_index.merge(df_table_filename, left_on='ID', right_on='ID')
df_index['Table Download Path'] = df_index['Table Name']\
    .apply(lambda x: '/tables/{}.zip'.format(x))

In [346]:
df_index.tail()

,Index,Title,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,Data ID,PDF Download URL,Application Type (NEB Act),Pipeline Location,Hearing order,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,ESA Section(s) Topics,CSV Download URL,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,Download folder name,Zipped Project Link,bad_csv,filename,ID,"Landscape, terrain, and weather",Soil,Plants,Water,Fish,Wetlands,Wildlife,Species at Risk,Greenhouse gas emissions,Air emissions,Noise,Electricity and electromagnetism,Proximity to people,"Archaeological, paleontological, historical, and culturally significant sites and resources",Human access to boats and waterways,"Indigenous land, water, and air use",Impact to social and cultural well-being,Impact to human health and viewscapes,"Social, cultural, economic infrastructure and services",Economic Offsets and Impact,Environmental Obligations,Treaty and Indigenous Rights,Project Download Path,Table Name,Table Download Path
27681,39045,Table 7.3 Characterization of Residual Effects on Wildlife and Wildlife Habitat,Table,Application for the NGTL West Path Delivery 2023 Project,West Path Delivery 2023,10/22/2020,NOVA Gas Transmission Ltd.,Gas,C09063-11 NGTL WPD 2023_ESA_Part 2_Sections 7 to 18 - A7J7C9,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3974306,A7J7C9,4003703,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/4003703,Large Projects (over 40 km),Alberta,GH-002-2020,Stantec Consulting Ltd,Applied,NaN,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3968941,NaN,"7.0 ASSESSMENT OF EFFECTS ON WILDLIFE AND WILDLIFE HABITAT, 8.0 ASSESSMENT OF EFFECTS ON AQUATIC...",NaN,NaN,http://www.cer-rec.gc.ca/esa-ees/wstpthdlvr2023/wstpthdlvr2023_table-7-3-characterization-of-res...,9,612.0,13.74,Yes,wstpthdlvr2023,http://www.cer-rec.gc.ca/esa-ees/wstpthdlvr2023.zip,False,wstpthdlvr2023_table-7-3-characterization-of-residual-effects-on-_pt-1_pg-9_doc-num-A7J7C9.csv,13120,0,1,0,0,0,0,26,0,2,0,1,1,1,0,0,0,0,0,2,4,0,0,/projects/wstpthdlvr2023.zip,wstpthdlvr2023_table-7-3-characterization-of-residual-effects-on-_pt-1_pg-8_doc-num-A7J7C9,/tables/wstpthdlvr2023_table-7-3-characterization-of-residual-effects-on-_pt-1_pg-8_doc-num-A7J7...
27682,39041,Table 7.18 Land Cover Types Within the Lundbreck Section LAA – Application Case,Table,Application for the NGTL West Path Delivery 2023 Project,West Path Delivery 2023,10/22/2020,NOVA Gas Transmission Ltd.,Gas,C09063-11 NGTL WPD 2023_ESA_Part 2_Sections 7 to 18 - A7J7C9,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3974306,A7J7C9,4003703,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/4003703,Large Projects (over 40 km),Alberta,GH-002-2020,Stantec Consulting Ltd,Applied,NaN,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3968941,NaN,"7.0 ASSESSMENT OF EFFECTS ON WILDLIFE AND WILDLIFE HABITAT, 8.0 ASSESSMENT OF EFFECTS ON AQUATIC...",NaN,NaN,http://www.cer-rec.gc.ca/esa-ees/wstpthdlvr2023/wstpthdlvr2023_table-7-18-land-cover-types-withi...,83,612.0,13.74,Yes,wstpthdlvr2023,http://www.cer-rec.gc.ca/esa-ees/wstpthdlvr2023.zip,False,wstpthdlvr2023_table-7-18-land-cover-types-within-the-lundbreck-s_pt-1_pg-83_doc-num-A7J7C9.csv,13026,0,0,70,6,0,7,0,0,0,0,0,0,2,0,0,2,0,0,3,0,0,2,/projects/wstpthdlvr2023.zip,wstpthdlvr2023_table-7-18-land-cover-types-within-the-lundbreck-s_pt-1_pg-83_doc-num-A7J7C9,/tables/wstpthdlvr2023_table-7-18-land-cover-types-within-the-lundbreck-s_pt-1_pg-83_doc-num-A7J...
27683,39042,Table 7.18 Land Cover Types Within the Lundbreck Section LAA – Application Case,Table,Application for the NGTL West Path Delivery 2023 Project,West Path Delivery 2023,10/22/2020,NOVA Gas Transmission Ltd.,Gas,C09063-11 NGTL WPD 2023_ESA_Part 2_Sections 7 to 18 - A7J7C9,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3974306,A7J7C9,4003703,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/4003703,Large Projects (over 

In [347]:
# Remove ',All' from Pipeline Location, ESA Section(s) Topics
df_index['Pipeline Location'] = df_index['Pipeline Location']\
    .apply(lambda x: ', '.join([location for location in x.split(', ') if location != 'All']))
df_index['ESA Section(s) Topics'] = df_index['ESA Section(s) Topics']\
    .apply(lambda x: ', '.join([section for section in x.split(', ') if section != 'All']) if type(x) is str else x)

In [348]:
# Update ESA Download URLs
df_index['ESA Folder URL'] = df_index['ESA Folder URL'].apply(lambda x: x.replace('LoadResult', 'View'))

In [349]:
# Prepare a list of column names for the final index files
columns_index = [col for col in df_index.columns.to_list() if col not in (
    'Table Name', 'CSV Download URL', 'Download folder name', 'Zipped Project Link', 'Unnamed: 0',
    'Unnamed: 0.1', 'Index', 'filename', 'old_filename', 'bad_csv')]

In [350]:
len(df_index)

27686

In [351]:
# =============================== Create Master Index File ===============================
# Added figures back to alpha index file
df_index_with_figure = pd.read_excel('//luxor/data/Branch/Environmental Baseline Data/Version 4 - Final/Indices/ESA_website_ENG.xlsx')
df_new_projects_with_figure = pd.read_csv("G:/ESA_downloads/phase_2_download_files/ESA_downloads/ESA_website_ENG_new_projects.csv")
figure_columns = columns_index.copy()
figure_columns.remove('ID')
figure_columns.remove('Project Download Path')
figure_columns.remove('Table Download Path')
df_figure = df_index_with_figure[df_index_with_figure['Content Type'] == 'Figure'][figure_columns]
df_figure['Application Filing Date'] = pd.to_datetime(df_figure['Application Filing Date'], format='%Y-%m-%d %H:%M:%S')
df_figure['Application Filing Date'] = df_figure['Application Filing Date'].dt.strftime('%Y-%m-%d')
df_new_projects_with_figure = df_new_projects_with_figure.reindex(df_new_projects_with_figure.columns.tolist() + labels_list, axis=1)
df_new_projects_with_figure['Application Filing Date'] = pd.to_datetime(df_new_projects_with_figure['Application Filing Date'], format='%m/%d/%Y')
df_new_projects_with_figure['Application Filing Date'] = df_new_projects_with_figure['Application Filing Date'].dt.strftime('%Y-%m-%d')
df_new_figure = df_new_projects_with_figure[df_new_projects_with_figure['Content Type'] == 'Figure'][figure_columns]
df_figure = df_figure.append(df_new_figure)
df_figure['ID'] = df_figure.index + df_index_raw['ID'].max() + 1


In [352]:
df_figure.tail(2)

,Title,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,PDF Download URL,Application Type (NEB Act),Pipeline Location,Hearing order,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,ESA Section(s) Topics,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,"Landscape, terrain, and weather",Soil,Plants,Water,Fish,Wetlands,Wildlife,Species at Risk,Greenhouse gas emissions,Air emissions,Noise,Electricity and electromagnetism,Proximity to people,"Archaeological, paleontological, historical, and culturally significant sites and resources",Human access to boats and waterways,"Indigenous land, water, and air use",Impact to social and cultural well-being,Impact to human health and viewscapes,"Social, cultural, economic infrastructure and services",Economic Offsets and Impact,Environmental Obligations,Treaty and Indigenous Rights,ID
642,"Figure 14.1 Locations of Nearest Populated Land Base of Engaged Indigenous Groups, Turner Valley...",Figure,Application for the NGTL West Path Delivery 2023 Project,West Path Delivery 2023,2020-10-22,NOVA Gas Transmission Ltd.,Gas,C09063-11 NGTL WPD 2023_ESA_Part 2_Sections 7 to 18 - A7J7C9,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3974306,A7J7C9,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/4003703,Large Projects (over 40 km),Alberta,GH-002-2020,Stantec Consulting Ltd,Applied,NaN,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3968941,NaN,"7.0 ASSESSMENT OF EFFECTS ON WILDLIFE AND WILDLIFE HABITAT, 8.0 ASSESSMENT OF EFFECTS ON AQUATIC...",NaN,NaN,499,612.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16299
643,"Figure 14.2 Locations of Nearest Populated Land Base of Engaged Indigenous Groups, Lundbreck Sec...",Figure,Application for the NGTL West Path Delivery 2023 Project,West Path Delivery 2023,2020-10-22,NOVA Gas Transmission Ltd.,Gas,C09063-11 NGTL WPD 2023_ESA_Part 2_Sections 7 to 18 - A7J7C9,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3974306,A7J7C9,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/4003703,Large Projects (over 40 km),Alberta,GH-002-2020,Stantec Consulting Ltd,Applied,NaN,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3968941,NaN,"7.0 ASSESSMENT OF EFFECTS ON WILDLIFE AND WILDLIFE HABITAT, 8.0 ASSESSMENT OF EFFECTS ON AQUATIC...",NaN,NaN,500,612.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16300


In [353]:
# Add bad tables
bad_table_columns = columns_index.copy()
bad_table_columns.remove('Project Download Path')
bad_table_columns.remove('Table Download Path')
df_table_bad = df_index_raw[df_index_raw['bad_csv']].sort_values(['ID', 'PDF Page Number']).groupby('ID').first()\
    .reset_index()[bad_table_columns]
df_table_bad['Good Quality'] = False

In [354]:
# Concatenate all tables
df_table = df_index.sort_values(['ID', 'PDF Page Number']).groupby('ID').first()\
    .reset_index()[columns_index]
df_table['Good Quality'] = True

In [355]:
df_index_new = pd.concat([df_figure, df_table, df_table_bad])

# Remove ',All' from Pipeline Location, ESA Section(s) Topics
df_index_new['Pipeline Location'] = df_index_new['Pipeline Location']\
    .apply(lambda x: ', '.join([location for location in x.split(', ') if location != 'All']))
df_index_new['ESA Section(s) Topics'] = df_index_new['ESA Section(s) Topics']\
    .apply(lambda x: ', '.join([section for section in x.split(', ') if section != 'All']) if type(x) is str else x)
# Update ESA Download URls
df_index_new['ESA Folder URL'] = df_index_new['ESA Folder URL'].apply(lambda x: x.replace('LoadResult', 'View'))

In [356]:
df_index_new[labels_list] = df_index_new[labels_list].astype('Int64')

In [357]:
df_index_new.reset_index(drop=True, inplace=True)

In [358]:
df_index_new.shape

(24929, 52)

In [359]:
df_index_new.tail()

,Title,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,PDF Download URL,Application Type (NEB Act),Pipeline Location,Hearing order,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,ESA Section(s) Topics,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,"Landscape, terrain, and weather",Soil,Plants,Water,Fish,Wetlands,Wildlife,Species at Risk,Greenhouse gas emissions,Air emissions,Noise,Electricity and electromagnetism,Proximity to people,"Archaeological, paleontological, historical, and culturally significant sites and resources",Human access to boats and waterways,"Indigenous land, water, and air use",Impact to social and cultural well-being,Impact to human health and viewscapes,"Social, cultural, economic infrastructure and services",Economic Offsets and Impact,Environmental Obligations,Treaty and Indigenous Rights,ID,Project Download Path,Table Download Path,Good Quality
24924,Table A8-1 Spill Reporting Authorities (cont�d),Table,Application for the Keystone XL Pipeline,Keystone XL,2/27/2009,TransCanada Keystone Pipeline GP Ltd.,Oil,B-2gg - KXL ESA - Appendix A Part 1 of 14 - A1I9X2,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/549185,A1I9X2,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/549421,Large Projects (over 40 km),"Alberta, Saskatchewan",OH-1-2009,Jacques Whitford AXYS Ltd,Approved,OC-056,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/550305,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/604643,Appendix A: TransCanada Keystone Environmental\nProtection Plan,33.0,Environment Protection Plan,103,118.0,3.59,No,0,0,3,2,1,0,0,0,0,0,0,1,0,0,0,1,2,1,12,0,0,3,14773,NaN,NaN,False
24925,Table A8-2 Release Reporting Criteria,Table,Application for the Keystone XL Pipeline,Keystone XL,2/27/2009,TransCanada Keystone Pipeline GP Ltd.,Oil,B-2gg - KXL ESA - Appendix A Part 1 of 14 - A1I9X2,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/549185,A1I9X2,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/549421,Large Projects (over 40 km),"Alberta, Saskatchewan",OH-1-2009,Jacques Whitford AXYS Ltd,Approved,OC-056,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/550305,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/604643,Appendix A: TransCanada Keystone Environmental\nProtection Plan,33.0,Environment Protection Plan,104,118.0,3.59,No,0,1,1,3,0,0,0,0,6,6,0,0,0,0,2,0,0,0,0,0,5,1,14774,NaN,NaN,False
24926,Table A9-2 Site Risk Levels,Table,Application for the Keystone XL Pipeline,Keystone XL,2/27/2009,TransCanada Keystone Pipeline GP Ltd.,Oil,B-2gg - KXL ESA - Appendix A Part 1 of 14 - A1I9X2,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/549185,A1I9X2,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/549421,Large Projects (over 40 km),"Alberta, Saskatchewan",OH-1-2009,Jacques Whitford AXYS Ltd,Approved,OC-056,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/550305,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/604643,Appendix A: TransCanada Keystone Environmental\nProtection Plan,33.0,Environment Protection Plan,110,118.0,3.59,No,0,2,4,2,0,0,0,4,0,0,0,0,1,0,1,0,1,1,2,1,0,0,14775,NaN,NaN,False
24927,Table B-1: Contacts Nelson Jalotjot Project Manager TransCanada PipeLines Limited 450 - 1st Stre...,Table,Application for the Leismer to Kettle River Crossover,Leismer to Kettle River Crossover,7/15/2011,NOVA Gas Transmission Ltd.,Gas,B1-19 - ESA Appendix A Part 1 of 2 - A2A6Q5,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/702711,A2A6Q5,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/702730,Large Projects (over 40 km),Alberta,GH-004-2011,Golder Associates,Operating,GC-120,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/704296,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/833910,Appendix A: Environmental Protection Plan,4.0,Environment Protection Plan,61,104.0,2.33,No,1,0,17,19,11,0,9,11,2,1,5,0,1,12,25,12,4,1,38,9,8,18,14838,NaN,NaN,False
24928,"Tables 4.12.1, 4.12.2, and 4.12.3 provide a summary of select l

In [360]:
# Create a new folder as the destination for downloading files
new_folder = os.path.join('//luxor/data/Board/ESA_downloads/', 'ESA_with_VCs')
if not os.path.exists(new_folder):
    os.mkdir(new_folder)

In [361]:
# Export alpha index
df_index_new.to_csv(os.path.join(new_folder, 'ESA_website_ENG.csv'), index=False)

In [276]:
zip_folders = "G:\ESA_downloads\download_Bingjie_Mar262021"
new_folder_projects = os.path.join(zip_folders, 'projects')
new_folder_tables = os.path.join(zip_folders, 'tables')

In [277]:
# Quality check

df_index_new = pd.read_csv(os.path.join(new_folder, 'ESA_website_ENG.csv'))

for project_path in df_index_new['Project Download Path'].unique():
    if type(project_path) is str:
        project_folder = project_path.split('/')[-1]
        if project_folder not in os.listdir(new_folder_projects):
            print('Missing project folder: {}'.format(project_folder))

for table_path in df_index_new['Table Download Path']:
    if type(table_path) is str:
        table_folder = table_path.split('/')[-1]
        if table_folder not in os.listdir(new_folder_tables):
            print('Missing table folder: {}'.format(table_folder))

df_index_new['table missing'] = df_index_new['Table Download Path']\
    .apply(lambda x: x.split('/')[-1] not in os.listdir(new_folder_tables) if type(x) is str else x)

for table_id in df_index_new[df_index_new['table missing'] == True]['ID']:
    print(table_id)
    bundle_for_table(df_index, table_id, new_folder_tables, csv_file_folder, columns_index, readme_table_filepath)

for table_path in df_index_new[df_index_new['table missing'] == True]['Table Download Path']:
    if type(table_path) is str:
        table_folder = table_path.split('/')[-1]
        if table_folder not in os.listdir(new_folder_tables):
            print('Missing table folder: {}'.format(table_folder))


C:\Users\ThibJacq\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (51) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Missing project folder: wstpthdlvr2022.zip
Missing project folder: wstpthdlvr2023.zip
Missing table folder: wstpthdlvr2022_table-1-timeframes-for-submitting-copy-1-of-the-ma_pt-1_pg-204_doc-num-A7I8X1.zip
Missing table folder: wstpthdlvr2022_table-1a-project-specific-mitigation-measures-rav_pt-1_pg-29_doc-num-A7I8X1.zip
Missing table folder: wstpthdlvr2022_table-1b-project-specific-mitigation-measures-abc_pt-1_pg-37_doc-num-A7I8X1.zip
Missing table folder: wstpthdlvr2022_table-2-potential-access-management-measures-type-_pt-1_pg-221_doc-num-A7I8X1.zip
Missing table folder: wstpthdlvr2022_table-2a-summary-of-watercourse-crossings-raven-r_pt-1_pg-63_doc-num-A7I8X1.zip
Missing table folder: wstpthdlvr2022_table-2b-summary-of-watercourse-crossings-abc-sec_pt-1_pg-65_doc-num-A7I8X1.zip
Missing table folder: tmx_table-3-17-summary-of-exposure-limits-selected-for_pt-1_pg-50_doc-num-A3S2L1.zip
Missing table folder: 2017ngtl_table-3-4-12-fort-mcmurray-m�tis-local-no-1935-tlr_pt-1_pg-80_doc-num-

NameError: name 'bundle_for_table' is not defined

In [ ]:

# Check if certain attributes stay the same for projects
df = pd.read_csv('G:/ESA_downloads/download_Bingjie_Mar262021/ESA_website_ENG.csv')
df_fra = pd.read_csv('G:/ESA_downloads/download_Bingjie_May062021_fra/ESA_website_FRA.csv', encoding='latin-1')


In [ ]:

df_project = df.groupby(['Application Short Name'])[[
    'Application Name',
    'Application Filing Date',
    'Company Name',
    'Commodity',
    'Application Type (NEB Act)',
    'Pipeline Location',
    'Hearing order',
    'Consultant Name',
    'Pipeline Status',
    'Application URL',
    'Project Download Path'
    ]].nunique()

df_project_fra = df_fra.groupby(['Nom abrégé de la demande'])[[
    'Nom de la demande',
    'Dépôt de la demande',
    'Nom de la société',
    'Produit de base',
    'Type de demande (Loi sur l\'Office national de l\'énergie)',
    'Emplacement du pipeline',
    'Ordonnance d\'audience',
    'Nom du consultant',
    'État d\'avancement',
    'URL de la demande',
    "Chemin d'accès pour télécharger le projet"
]].nunique()
